### Wine rating classification

In [64]:
import importlib

import mysklearn.myclassifiers
from mysklearn.myclassifiers import MyKNeighborsClassifier
from mysklearn.myclassifiers import MyDecisionTreeClassifier
from mysklearn.myclassifiers import MyNaiveBayesClassifier

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

In [ ]:
mpt = MyPyTable()
filename = "input_data/WineRatings.csv"
mpt.load_from_file(filename)
# set the Name column because didn't load properly
mpt.column_names[0] = 'Name'

['Red', 'Rose', 'White', 'Sparkling']


Investigate missing values, missing values in the dataset are denoted as "N.V."

In [66]:
# count the missing values in the dataset
count = myutils.count_nv(mpt.data)
name_col = mpt.get_column('Name')
dataset_len = len(name_col)
print(f'Out of {dataset_len} rows, there are {count} rows with missing data')

mpt.replace_missing_values_with_column_average('Price')
mpt.replace_missing_values_with_column_average('Rating')
mpt.replace_missing_values_with_column_average('Year')
summary_table = mpt.compute_summary_statistics(["Price", "Rating", "Year"])
summary_table.pretty_print()

Out of 13834 rows, there are 744 rows with missing data
attribute        min      max      mid         avg    median
-----------  -------  -------  -------  ----------  --------
Price           3.15  3410.79  1706.97    33.0249      15.95
Rating          2.2      4.9      3.55     3.86566      3.9
Year         1961     2020     1990.5   2015.53      2016


In [67]:
# convert to numerical
print('column names', mpt.column_names)
name_col = mpt.get_column('Name')
country_col = mpt.get_column('Country')
region_col = mpt.get_column('Region')
winery_col = mpt.get_column('Winery')
rating_col = mpt.get_column('Rating')
num_ratings_col = mpt.get_column('NumberOfRatings')
price_col = mpt.get_column('Price')
year_col = mpt.get_column('Year')
winestyle_col = mpt.get_column('Winestyle')

unique_names = myutils.get_unique_labels(name_col)
unique_countries = myutils.get_unique_labels(country_col)
unique_regions = myutils.get_unique_labels(region_col)
unique_wineries = myutils.get_unique_labels(winery_col)
unique_ratings = myutils.get_unique_labels(rating_col)
unique_num_ratings = myutils.get_unique_labels(num_ratings_col)
unique_prices = myutils.get_unique_labels(price_col)
unique_years = myutils.get_unique_labels(year_col)
unique_winestyles = myutils.get_unique_labels(winestyle_col)

num_unique_names = len(unique_names)
print(len(name_col))

print(f'There are {num_unique_names} unique names')

column names ['Name', 'Country', 'Region', 'Winery', 'Rating', 'NumberOfRatings', 'Price', 'Year', 'Winestyle']
13834
There are 10934 unique names


In [68]:
knn_classifier = MyKNeighborsClassifier()
